In [85]:
import sys
sys.path.append('../../')
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.02 ms (started: 2024-12-09 17:33:15 -08:00)


# Introduction
In this notebook, we are numerically calculating the average Hamiltonian produced by the pulse scheme proposed in [1] to mix a native XY Heisenberg Hamiltonian (symmetric exchange) into a new XY Hamiltonian with anti-symmetric exchange (the z-component of the DM interaction).


1. Nishad, N., Keselman, A., Lahaye, T., Browaeys, A. & Tsesses, S. Quantum simulation of generic spin-exchange models in Floquet-engineered Rydberg-atom arrays. Phys. Rev. A 108, 053318 (2023).


In [86]:
from models.spin_chain import LatticeGraph, DiagonEngine
import numpy as np
from scipy.linalg import expm
from __future__ import annotations

time: 552 µs (started: 2024-12-09 17:33:15 -08:00)


# Local Rotations
This paper uses local rotations in a size 4 unit cell (PBC, a ring of atoms L = 4N). Figure 2 in the paper has a good illustration. There are two basic pulses that rotate different sites with different amounts of phase to turn on/off the positive/negative DM/XY interactions. The evolution times between the local phase rotations determine the nature (mostly XY or mostly DM) of the average Hamiltonian in one cycle.

For now, we want to ignore the effect of finite pulse durations, so these functions return a total phase of rotation (an integrated delta function pulse) rather than a frequency/energy to be integrated over a finite time. We define another function that turns off the native Hamiltonian during these phase rotations (delta function pulses).

In [87]:
def DM_z_period4(t, i):
    phase = np.pi / 2 * (i % 4)
    if t == "+DM":
        return phase/2
    elif t == "-DM":
        return -phase/2
    else:
        return 0

def XY_z_period4(t, i):
    phase = np.pi - 3. * np.pi / 2 * (i % 4)
    if t == "+XY":
        return phase/2
    elif t == "-XY":
        return -phase/2
    else:
        return 0

def native(t, i, j):
    if t in ["+DM", "-DM", "+XY", "-XY"]:
        return 0
    else:
        return J/2

time: 765 µs (started: 2024-12-09 17:33:15 -08:00)


# Define the Hamiltonian
This is simply a list of terms in the (parametrized) Hamiltonian we want to numerically evolve in time. The first entry of each term is the operator as a string. The second element is the "strength" of the interaction, which here is parametrized by pulse type (as above) to make a piece-wise defined Hamiltonian (though one could consider defining a time-continuous parametrization, and we might do this later). The last element is the "connectivity" or "range" of the interactions defined either by a specific string or an inverse range value (alpha = 3 would be dipolar range, alpha = np.inf is an on-site interaction).

Here we initially consider the smallest system this pulse scheme allows, 4 atoms in a ring.

In [88]:
XY_terms = [['XX', native, 'nn'], ['yy', native, 'nn'],
             ['z', DM_z_period4, np.inf], ['z', XY_z_period4, np.inf]]
XY_graph = LatticeGraph.from_interactions(8, XY_terms, pbc=True)

time: 623 µs (started: 2024-12-09 17:33:15 -08:00)


## Define the Pulse Sequence
Here, the timing of the pulse sequence is defined. The paramList selects which pulses we want from our parametrization above, and dtList defines how long each of those pieces is evolved in time. Delta pulses act for zero time, so the only time evolution that happens in this example are the "free-evolution" times under the native Hamiltonian.

In [89]:
tD = 100e-9
tJ = 10e-9
tmJ = tJ
J = 10
paramList = ["nat", "+DM", "nat", "+XY", "nat", "-XY", "nat", "-DM", "nat"]
dtList = [tJ, 0, tD, 0, 2 * tmJ, 0, tD, 0, tJ]

time: 547 µs (started: 2024-12-09 17:33:15 -08:00)


This is our target Hamiltonian.

In [90]:
DM_terms = [['xy', 1/2, 'nn'], ['yx', -1/2, 'nn']]
DM_graph = LatticeGraph.from_interactions(8, DM_terms, pbc=True)

time: 428 µs (started: 2024-12-09 17:33:15 -08:00)


Under the hood, we use QuSpin to perform the time-evolution and calculate the Floquet/Average Hamiltonian. We compute the Hilbert-Schmidt overlap of the relevant Hamiltonians.

In [91]:
computation = DiagonEngine(XY_graph, unit_cell_length=4)
HF = computation.get_quspin_floquet_hamiltonian(paramList, dtList)
# print(HF)
H_XY = computation.get_quspin_hamiltonian(0)/J
XY_frobenius_loss = computation.frobenius_loss(HF, H_XY)
H_DM = DiagonEngine(DM_graph, unit_cell_length=4).get_quspin_hamiltonian(0)
DM_frobenius_loss = computation.frobenius_loss(HF, H_DM)
print("XY Frobenius loss:", XY_frobenius_loss)
print("DM Frobenius loss:", DM_frobenius_loss)

XY Frobenius loss: 0.99999556543913
DM Frobenius loss: 0.0
time: 3.7 s (started: 2024-12-09 17:33:15 -08:00)


We can compute another type of fidelity metric (used in the referenced paper) that approaches zero as the unitary evolution computed from the Hamiltonians approach perfect overlap. It is a much more sensitive measure of overlap than the Hilbert-Schmidt overlap we've defined above. This is probably a better metric for numerical optimization of the pulse sequence and therefore likely why it was the metric of choice in the reference paper.

In [92]:
T = sum(dtList) # the Floquet period
DM_norm_loss = computation.norm_identity_loss(HF, H_DM)
XY_norm_loss = computation.norm_identity_loss(HF, H_XY)
print("XY loss:", XY_norm_loss)
print("DM loss:", DM_norm_loss)

XY loss: 21.15128818710378
DM loss: 16.1871195134009
time: 132 ms (started: 2024-12-09 17:33:19 -08:00)


How should we think about the triangle of norm losses between each of the three Hamiltonians

In [93]:
cross_frobenius_loss = computation.frobenius_loss(H_DM, H_XY)
print("cross Frobenius loss (should be 1):", cross_frobenius_loss)
cross_norm_loss = computation.norm_identity_loss(H_DM, H_XY)
print("norm loss (should be large):", cross_norm_loss)

cross Frobenius loss (should be 1): 1.0
norm loss (should be large): 22.25245618715209
time: 82.5 ms (started: 2024-12-09 17:33:19 -08:00)


# Open Chain DM Exchange

In [124]:
def DM_z_period2(t, i):
    phase = np.pi/2 * (i % 2)
    if t == "+DM":
        return phase/2
    elif t == "-DM":
        return -phase/2
    else:
        return 0

def XY_z_period2(t, i):
    phase = np.pi/2 * ((i+1) % 2)
    if t == "+XY":
        return phase/2
    elif t == "-XY":
        return -phase/2
    else:
        return 0

def native(t, i, j):
    if t in ["+DM", "-DM", "+XY", "-XY"]:
        return 0
    else:
        return J/2

time: 872 µs (started: 2024-12-09 17:35:01 -08:00)


In [125]:
XY_terms = [['XX', native, 'nn'], ['yy', native, 'nn'],
             ['z', DM_z_period4, np.inf], ['z', XY_z_period4, np.inf]]
XY_graph = LatticeGraph.from_interactions(4, XY_terms)

time: 444 µs (started: 2024-12-09 17:35:01 -08:00)


In [126]:
DM_terms = [['xy', 1/2, 'nn'], ['yx', -1/2, 'nn']]
DM_graph = LatticeGraph.from_interactions(4, DM_terms)

time: 427 µs (started: 2024-12-09 17:35:01 -08:00)


In [127]:
tD = 100e-9
tJ = 10e-9
tmJ = tJ
J = 1000
paramList = ["nat", "+DM", "nat", "+XY", "nat", "-XY", "nat", "-DM", "nat"]
dtList = [tJ, 0, tD, 0, 2 * tmJ, 0, tD, 0, tJ]

time: 481 µs (started: 2024-12-09 17:35:01 -08:00)


In [128]:
computation = DiagonEngine(XY_graph, unit_cell_length=2)
HF = computation.get_quspin_floquet_hamiltonian(paramList, dtList)
# print(HF)
H_XY = computation.get_quspin_hamiltonian(0)/J
H_DM = DiagonEngine(DM_graph, unit_cell_length=2).get_quspin_hamiltonian(0)
DM_norm_loss = computation.norm_identity_loss(HF, H_DM)
XY_norm_loss = computation.norm_identity_loss(HF, H_XY)
print("XY loss:", XY_norm_loss)
print("DM loss:", DM_norm_loss)

XY loss: 5.560097930912807
DM loss: 2.696477576742332
time: 202 ms (started: 2024-12-09 17:35:02 -08:00)


In [129]:
XY_frobenius_loss = computation.frobenius_loss(HF, H_XY)
DM_frobenius_loss = computation.frobenius_loss(HF, H_DM)
print("XY Frobenius loss:", XY_frobenius_loss)
print("DM Frobenius loss:", DM_frobenius_loss)

XY Frobenius loss: 0.999974179634162
DM Frobenius loss: 0.0
time: 2.82 ms (started: 2024-12-09 17:35:30 -08:00)


In [130]:
print(H_DM)

static mat: 
<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 24 stored elements and shape (16, 16)>
  Coords	Values
  (1, 2)	1j
  (2, 1)	-1j
  (2, 4)	1j
  (3, 5)	1j
  (4, 2)	-1j
  (4, 8)	1j
  (5, 3)	-1j
  (5, 6)	1j
  (5, 9)	1j
  (6, 5)	-1j
  (6, 10)	1j
  (7, 11)	1j
  (8, 4)	-1j
  (9, 5)	-1j
  (9, 10)	1j
  (10, 6)	-1j
  (10, 9)	-1j
  (10, 12)	1j
  (11, 7)	-1j
  (11, 13)	1j
  (12, 10)	-1j
  (13, 11)	-1j
  (13, 14)	1j
  (14, 13)	-1j


dynamic:

time: 956 µs (started: 2024-12-09 17:35:48 -08:00)


In [136]:
from scipy import sparse
print(sparse.csr_matrix(np.rint(HF/J)))

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 24 stored elements and shape (16, 16)>
  Coords	Values
  (1, 2)	(-0-1j)
  (2, 1)	(-0+1j)
  (2, 4)	-1j
  (3, 5)	(-0-1j)
  (4, 2)	(-0+1j)
  (4, 8)	(-0-1j)
  (5, 3)	1j
  (5, 6)	(-0-1j)
  (5, 9)	(-0-1j)
  (6, 5)	(-0+1j)
  (6, 10)	(-0-1j)
  (7, 11)	(-0-1j)
  (8, 4)	(-0+1j)
  (9, 5)	(-0+1j)
  (9, 10)	(-0-1j)
  (10, 6)	(-0+1j)
  (10, 9)	(-0+1j)
  (10, 12)	(-0-1j)
  (11, 7)	(-0+1j)
  (11, 13)	-1j
  (12, 10)	(-0+1j)
  (13, 11)	(-0+1j)
  (13, 14)	(-0-1j)
  (14, 13)	(-0+1j)
time: 1.11 ms (started: 2024-12-09 17:39:40 -08:00)
